# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 8 2022 8:30AM,245904,10,SONSB0001927,"Nextsource Biotechnology, LLC",Released
1,Sep 7 2022 4:19PM,245909,12,HH6720F,Hush Hush,Released
2,Sep 7 2022 4:19PM,245908,22,626674,"NBTY Global, Inc.",Released
3,Sep 7 2022 4:19PM,245789,21,OTM100940443,"NBTY Global, Inc.",Released
4,Sep 7 2022 4:19PM,245907,22,626673,"NBTY Global, Inc.",Released
5,Sep 7 2022 4:17PM,245906,19,60017417,"GUSA Granules USA, Inc.",Released
6,Sep 7 2022 4:16PM,245650,21,626225,"NBTY Global, Inc.",Released
7,Sep 7 2022 4:13PM,245795,21,626535,"NBTY Global, Inc.",Released
8,Sep 7 2022 4:12PM,245905,19,60017416,"GUSA Granules USA, Inc.",Executing
9,Sep 7 2022 4:12PM,245730,21,625945,"NBTY Global, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
54,245905,Executing,1
55,245906,Released,1
56,245907,Released,1
57,245908,Released,1
58,245909,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
245905,NaN,1.0,NaN
245906,NaN,NaN,1.0
245907,NaN,NaN,1.0
245908,NaN,NaN,1.0
245909,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
245648,0.0,0.0,1.0
245649,0.0,0.0,1.0
245650,0.0,0.0,1.0
245714,22.0,2.0,2.0
245726,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
245648,0,0,1
245649,0,0,1
245650,0,0,1
245714,22,2,2
245726,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,245648,0,0,1
1,245649,0,0,1
2,245650,0,0,1
3,245714,22,2,2
4,245726,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,245648,,,1
1,245649,,,1
2,245650,,,1
3,245714,22,2,2
4,245726,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 8 2022 8:30AM,245904,10,"Nextsource Biotechnology, LLC"
1,Sep 7 2022 4:19PM,245909,12,Hush Hush
2,Sep 7 2022 4:19PM,245908,22,"NBTY Global, Inc."
3,Sep 7 2022 4:19PM,245789,21,"NBTY Global, Inc."
4,Sep 7 2022 4:19PM,245907,22,"NBTY Global, Inc."
5,Sep 7 2022 4:17PM,245906,19,"GUSA Granules USA, Inc."
6,Sep 7 2022 4:16PM,245650,21,"NBTY Global, Inc."
7,Sep 7 2022 4:13PM,245795,21,"NBTY Global, Inc."
8,Sep 7 2022 4:12PM,245905,19,"GUSA Granules USA, Inc."
9,Sep 7 2022 4:12PM,245730,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 8 2022 8:30AM,245904,10,"Nextsource Biotechnology, LLC",,,1
1,Sep 7 2022 4:19PM,245909,12,Hush Hush,,,1
2,Sep 7 2022 4:19PM,245908,22,"NBTY Global, Inc.",,,1
3,Sep 7 2022 4:19PM,245789,21,"NBTY Global, Inc.",,,1
4,Sep 7 2022 4:19PM,245907,22,"NBTY Global, Inc.",,,1
5,Sep 7 2022 4:17PM,245906,19,"GUSA Granules USA, Inc.",,,1
6,Sep 7 2022 4:16PM,245650,21,"NBTY Global, Inc.",,,1
7,Sep 7 2022 4:13PM,245795,21,"NBTY Global, Inc.",,,1
8,Sep 7 2022 4:12PM,245905,19,"GUSA Granules USA, Inc.",,1,
9,Sep 7 2022 4:12PM,245730,21,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 8 2022 8:30AM,245904,10,"Nextsource Biotechnology, LLC",1,,
1,Sep 7 2022 4:19PM,245909,12,Hush Hush,1,,
2,Sep 7 2022 4:19PM,245908,22,"NBTY Global, Inc.",1,,
3,Sep 7 2022 4:19PM,245789,21,"NBTY Global, Inc.",1,,
4,Sep 7 2022 4:19PM,245907,22,"NBTY Global, Inc.",1,,
5,Sep 7 2022 4:17PM,245906,19,"GUSA Granules USA, Inc.",1,,
6,Sep 7 2022 4:16PM,245650,21,"NBTY Global, Inc.",1,,
7,Sep 7 2022 4:13PM,245795,21,"NBTY Global, Inc.",1,,
8,Sep 7 2022 4:12PM,245905,19,"GUSA Granules USA, Inc.",,1,
9,Sep 7 2022 4:12PM,245730,21,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 8 2022 8:30AM,245904,10,"Nextsource Biotechnology, LLC",1,,
1,Sep 7 2022 4:19PM,245909,12,Hush Hush,1,,
2,Sep 7 2022 4:19PM,245908,22,"NBTY Global, Inc.",1,,
3,Sep 7 2022 4:19PM,245789,21,"NBTY Global, Inc.",1,,
4,Sep 7 2022 4:19PM,245907,22,"NBTY Global, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 8 2022 8:30AM,245904,10,"Nextsource Biotechnology, LLC",1.0,NaN,NaN
1,Sep 7 2022 4:19PM,245909,12,Hush Hush,1.0,NaN,NaN
2,Sep 7 2022 4:19PM,245908,22,"NBTY Global, Inc.",1.0,NaN,NaN
3,Sep 7 2022 4:19PM,245789,21,"NBTY Global, Inc.",1.0,NaN,NaN
4,Sep 7 2022 4:19PM,245907,22,"NBTY Global, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 8 2022 8:30AM,245904,10,"Nextsource Biotechnology, LLC",1.0,0.0,0.0
1,Sep 7 2022 4:19PM,245909,12,Hush Hush,1.0,0.0,0.0
2,Sep 7 2022 4:19PM,245908,22,"NBTY Global, Inc.",1.0,0.0,0.0
3,Sep 7 2022 4:19PM,245789,21,"NBTY Global, Inc.",1.0,0.0,0.0
4,Sep 7 2022 4:19PM,245907,22,"NBTY Global, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3933485,146.0,26.0,7.0
12,737648,3.0,0.0,0.0
15,983278,66.0,20.0,58.0
16,491743,6.0,0.0,0.0
19,1229510,4.0,3.0,0.0
20,1229390,24.0,11.0,9.0
21,2211512,8.0,1.0,0.0
22,737712,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3933485,146.0,26.0,7.0
1,12,737648,3.0,0.0,0.0
2,15,983278,66.0,20.0,58.0
3,16,491743,6.0,0.0,0.0
4,19,1229510,4.0,3.0,0.0
5,20,1229390,24.0,11.0,9.0
6,21,2211512,8.0,1.0,0.0
7,22,737712,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,146.0,26.0,7.0
1,12,3.0,0.0,0.0
2,15,66.0,20.0,58.0
3,16,6.0,0.0,0.0
4,19,4.0,3.0,0.0
5,20,24.0,11.0,9.0
6,21,8.0,1.0,0.0
7,22,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,146.0
1,12,Released,3.0
2,15,Released,66.0
3,16,Released,6.0
4,19,Released,4.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21,22
Status,,,,,,,,
Completed,7.0,0.0,58.0,0.0,0.0,9.0,0.0,0.0
Executing,26.0,0.0,20.0,0.0,3.0,11.0,1.0,0.0
Released,146.0,3.0,66.0,6.0,4.0,24.0,8.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21,22
0,Completed,7.0,0.0,58.0,0.0,0.0,9.0,0.0,0.0
1,Executing,26.0,0.0,20.0,0.0,3.0,11.0,1.0,0.0
2,Released,146.0,3.0,66.0,6.0,4.0,24.0,8.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21,22
0,Completed,7.0,0.0,58.0,0.0,0.0,9.0,0.0,0.0
1,Executing,26.0,0.0,20.0,0.0,3.0,11.0,1.0,0.0
2,Released,146.0,3.0,66.0,6.0,4.0,24.0,8.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()